## Class 5

## Assignment

Download 200 quarterly call transcripts from www.fool.com

Don't forget to wait .1 seconds between each download

Use the ticker as a unique identifier for this section

Put in dataframe

Extract the prepared remarks

Calculate the same scores for each document that we've been calculating

### Captions:
    "### ": Expanation of the code block in general;
    "## ": Explanation comments in code
    "## ->": Variables that need to be changed according to the unique specifications of whoever is applying this code.
    "# ": Commented code blocks that I did not use for the final execution but that are interesting for the developer


In [2]:
### Module imports necessary for the code to function

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import requests
## Helper is the file created to put all the functions related to calculating text metrics, with the purpose of not having to repeat the same code.
## This functions will probably be used in other applications.
from Helper import Helper

ModuleNotFoundError: No module named 'bs4'

#### 1a. Finding the page in with the links of transcripts are organized

In [3]:
### Finding the pages with all transcripts in www.fool.com


## Main page of transcripts
to_go_url = r'https://www.fool.com/earnings-call-transcripts/'

## Example of pages (load more), to get 200 transcripts we need to go through some pages.
## All of the pages starts with "?page=" and the number of pages
to_go_url2 = r'https://www.fool.com/earnings-call-transcripts/?page=2'

## A test to find the link of each page. In this case, for each number in 0 to 10:
for i in range(10):
## The code get the link go_to_url and join with ?page= and the number that is in loop;
  print (to_go_url + f"?page={i+1}")

https://www.fool.com/earnings-call-transcripts/?page=1
https://www.fool.com/earnings-call-transcripts/?page=2
https://www.fool.com/earnings-call-transcripts/?page=3
https://www.fool.com/earnings-call-transcripts/?page=4
https://www.fool.com/earnings-call-transcripts/?page=5
https://www.fool.com/earnings-call-transcripts/?page=6
https://www.fool.com/earnings-call-transcripts/?page=7
https://www.fool.com/earnings-call-transcripts/?page=8
https://www.fool.com/earnings-call-transcripts/?page=9
https://www.fool.com/earnings-call-transcripts/?page=10


### 1b. Testing the extration of page with transcriptions links"

In [4]:
### Opening the page with all transcriptions and get the content

## Test page
testSite = r'https://www.fool.com/earnings-call-transcripts/?page=1'

## Header is used to send a request.
head={"User-Agent":"Mozilla/5.0(Windows NT 10.0; Win64; x64)"}

## Creating a Request object with the header and test page.
req = Request(testSite, headers=head)

## Content is getting the content of the page.
## The function urlopen() is used to open the page with all html tags
content = str(urlopen(req).read(), 'utf-8')

## Seeing how content looks like
print(content)



<!DOCTYPE html>
<html lang="en">
<head>


<script type="text/javascript" src="https://cdn.cookielaw.org/consent/02abb198-81a8-49e5-a9b1-f69a5dd9c039/OtAutoBlock.js"></script>
<script src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" type="text/javascript" charset="UTF-8" data-domain-script="02abb198-81a8-49e5-a9b1-f69a5dd9c039"></script>

<script>
function OptanonWrapper(){var t;window.infotrackInitialized||void 0===window.Infotrack?window.Infotrack.setSegmentIntegrations():(window.infotrackInitialized=!0,Infotrack.load("https://g.foolcdn.com/mms/resources/js/infotrack-ot_min.js"),t=t||0,setTimeout(function(){window.Infotrack.initialize("usmf")},t))}window.top!=window&&(window.analytics=window.top.analytics,window.inIframe=!0);var segmentKey="CL257pIDks5ukrbYAs7Ez3fkLz1bWVnJ",segmentSnippetVersion="4.15.2",getSegmentUrl=function(t){return t=t||window.segmentKey,("https:"===document.location.protocol?"https://":"http://")+"evs.pink-boat.fool.com/analytics.js/v1/"+t+"/analyt

In [5]:
### Finding the pages for each company

## Creating a beultifulsoup object.
soup = BeautifulSoup(content)

## Empty List to put a chunk of links from transcription pages, similar with this: "2023/10/05/constellation-brands-stz-q2-2024-earnings-call-tra/"
callRefs = []

## O loop searh for tags <a> in content, to each a:
## Search for all classes with name 'flex-shrink-0 w-1/3 mr-16px sm:w-auto' thet has the attribute href, witch has a chunk link that we put in the list
for a in soup.find_all('a', {'class': 'flex-shrink-0 w-1/3 mr-16px sm:w-auto'}, href=True):
## When locates the link, he append in the list. (Printed to check if is working)
  callRefs.append(a['href'])
  print(a['href'])

## Printed to estimate how many links are in one page.
print("A lista possui:",len(callRefs),"itens")

/earnings/call-transcripts/2023/10/13/jpmorgan-chase-jpm-q3-2023-earnings-call-transcrip/
/earnings/call-transcripts/2023/10/12/commercial-metals-cmc-q4-2023-earnings-call-transc/
/earnings/call-transcripts/2023/10/12/delta-air-lines-dal-q3-2023-earnings-call-transcri/
/earnings/call-transcripts/2023/10/12/walgreens-boots-alliance-wba-q4-2023-earnings-call/
/earnings/call-transcripts/2023/10/11/voxx-international-corporation-voxx-q2-2024-earnin/
/earnings/call-transcripts/2023/10/05/constellation-brands-stz-q2-2024-earnings-call-tra/
/earnings/call-transcripts/2023/10/05/conagra-brands-cag-q1-2024-earnings-call-transcrip/
/earnings/call-transcripts/2023/10/04/tilray-brands-tlry-q1-2024-earnings-call-transcrip/
/earnings/call-transcripts/2023/10/04/carmax-kmx-q2-2024-earnings-call-transcript/
/earnings/call-transcripts/2023/10/03/oddity-tech-odd-q3-2023-earnings-call-transcript/
/earnings/call-transcripts/2023/10/03/mccormick-mkc-q3-2023-earnings-call-transcript/
/earnings/call-transcri

#### 1c. Testing the extration of Tickers and Prepared Remarks content

In [6]:
### Testing the open of a link and get the content.

## Get the link of main page with the transcription part (in this case for the first link).
testCall = r'https://www.fool.com' + callRefs[0]

## Creating a Request object with the header and test page.
req1 = Request(testCall, headers=head)

## Content is getting the content of the page.
## The function urlopen() is used to open the page with all html tags
all_page = str(urlopen(req1).read(), 'utf-8')

## Printing to check if is working.
print(all_page)



<!DOCTYPE html>
<html lang="en" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">
<head>


<script type="text/javascript" src="https://cdn.cookielaw.org/consent/02abb198-81a8-49e5-a9b1-f69a5dd9c039/OtAutoBlock.js"></script>
<script src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" type="text/javascript" charset="UTF-8" data-domain-script="02abb198-81a8-49e5-a9b1-f69a5dd9c039"></script>

<script>
function OptanonWrapper(){var t;window.infotrackInitialized||void 0===window.Infotrack?window.Infotrack.setSegmentIntegrations():(window.infotrackInitialized=!0,Infotrack.load("https://g.foolcdn.com/mms/resources/js/infotrack-ot_min.js"),t=t||0,setTimeout(function(){window.Infotrack.initialize("usmf")},t))}window.top!=window&&(window.analytics=window.top.analytics,window.inIframe=!0);var segmentKey="CL257pIDks5ukrbYAs7Ez3fkLz1bWVnJ",segmentSnippetVersion="4.15.2",getSegmentUrl=function(t){return t=t||window.segmentKey,("https:"===document.l

In [7]:
### Getting the company ticker

## Creating a beultifulsoup object.
soup = BeautifulSoup(all_page, 'html.parser')

## Finding the tag with the name "tickers"
tag = soup.find('meta', {'name': 'tickers'})

## If the result of tag is not empty
if tag != [] :
    ## Printing get the content of ticker
    ticker = tag['content']
    print(ticker)



JPM


In [12]:
### Extrecting Prepared Remarks section




## Function to extract text between tags, with content, start tag and end tag
def extract_text_between_tags(text, start_tag, end_tag):
## Regex expression that receives the initial tag, checks the content that comes after it, until the end tag 
## f"xxx{var}" means that we are passing a string, and a varible together.
## re.escape means that we are passing a string that contains special characters
    pattern = f"{re.escape(start_tag)}(.*?){re.escape(end_tag)}"
## matches finds the pattern, with the "re.DOTALL" and re.IGNORECASE option.
    matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
    return matches

## As verifyed, the page always starts with "<h2>Prepared Remarks:</h2>" and ends with "<h2>Questions &amp; Answers:</h2>"
start_tag = "<h2>Prepared Remarks:</h2>"
end_tag = "<h2>Questions &amp; Answers:</h2>"

## applying the function extract_text_between_tags in test page.
raw_prepared_remarks = extract_text_between_tags(all_page, start_tag, end_tag)

## if result is not empty, we create a beautifulsoup object with the content 
if raw_prepared_remarks != [] :
  soup1 = BeautifulSoup(raw_prepared_remarks[0])

## Getting the content without HTML tags, I put this separator because it seems to be a better way to not join words when the tags are eleiminated.
prepared_remarks = soup1.getText(separator=" ")

## Checking if is working
print(prepared_remarks)

    Operator Good morning, ladies and gentlemen. Welcome to JPMorgan Chase's third-quarter 2023 earnings call. This call is being recorded. [Operator instructions] At this time, I would like to turn the call over to JPMorgan Chase's chief financial officer, Jeremy Barnum; and their chairman and CEO, Jamie Dimon.   Mr. Dimon, please go ahead. Jamie Dimon  --  Chairman and Chief Executive Officer   Good morning, everybody. Before we start the actual call, I want to repeat something we just said on the press call. So before we get into discussions about third-quarter earnings, I just want to say how deeply saddened that we all are about the recent horrific attacks on Israel and the resulting bloodshed and more. Terrorism and hatred have no place in our civilized world, and all of our hearts here at JPMorgan Chase go out to all who are suffering. 10 stocks we like better than JPMorgan Chase When o ur analyst team has  a stock tip, it can pay to listen. After all, the newsletter they have r

#### 2. Applying the extration in 200 transcriptions and puting in a Dataframe

In [7]:
### Making a list with the chunk of links from transcription pages.

to_go_url = r'https://www.fool.com/earnings-call-transcripts/'

## Creating a Request object with the header and test page.
head={"User-Agent":"Mozilla/5.0(Windows NT 10.0; Win64; x64)"}


## To count how many links we got in the loop 
link_counter = 0
page_counter = 1

## Empty List to put a chunk of links from transcription pages, similar with this: "2023/10/05/constellation-brands-stz-q2-2024-earnings-call-tra/"
link_transcript_list = []




## While loop to get all links until the counter reach 200 (code was explained in test section)
while link_counter < 1000:
    main_page = to_go_url + f"?page={page_counter}"
    req = requests.get(main_page, headers=head)
    main_page_content = req.text
    soup = BeautifulSoup(main_page_content, 'html.parser')
    for a in soup.find_all('a', {'class': 'flex-shrink-0 w-1/3 mr-16px sm:w-auto'}, href=True):
        link = a['href']
        if link not in link_transcript_list:
            link_transcript_list.append(link)
    link_counter += len(link_transcript_list)
    page_counter += 1
    # Time sleep to avoid being blocked
    print("A lista possui:",len(link_transcript_list),"itens")
    time.sleep(0.40)



## Printing the length of the list to see if is working



A lista possui: 20 itens
A lista possui: 40 itens
A lista possui: 60 itens
A lista possui: 80 itens
A lista possui: 100 itens
A lista possui: 120 itens
A lista possui: 140 itens
A lista possui: 160 itens
A lista possui: 180 itens
A lista possui: 200 itens


In [8]:
### Extrating the content of each link 


## Creating a DataFrame
df = pd.DataFrame()



## For loop to get each link in the list and extract the ticker and prepared remarks content (code was explained in test section)
for link in link_transcript_list :
    link_transcription = r'https://www.fool.com' + link
    req = Request(link_transcription, headers=head)
    webpage_response = requests.get(link_transcription, headers=head)
    if webpage_response.status_code == 200:
        all_page = str(urlopen(req).read(), 'utf-8')
        soup = BeautifulSoup(all_page, 'html.parser')
        tag = soup.find('meta', {'name': 'tickers'})
        if tag != [] :
            ticker = tag['content']
            ## Here I concatenate the dataframe with the link, ticker, raw content
        df = pd.concat([df, pd.DataFrame({"Link": [link],"Ticker" : [ticker], "RawContent" : [all_page]}, index=[0])], ignore_index=True)
        ## Printing to see the extraction in real time
        print(link_transcription," works!")
    else :
        ## Printing the link that does not work
        print("WebPage response is not 200 "+link_transcription)
    ## Time sleep to avoid being blocked, be careful with indentation!
    time.sleep(0.20)



https://www.fool.com/earnings/call-transcripts/2023/10/13/oil-dri-of-america-odc-q4-2023-earnings-call-trans/  works!
https://www.fool.com/earnings/call-transcripts/2023/10/13/citigroup-c-q3-2023-earnings-call-transcript/  works!
https://www.fool.com/earnings/call-transcripts/2023/10/13/wells-fargo-wfc-q3-2023-earnings-call-transcript/  works!
https://www.fool.com/earnings/call-transcripts/2023/10/13/blackrock-blk-q3-2023-earnings-call-transcript/  works!
https://www.fool.com/earnings/call-transcripts/2023/10/13/unitedhealth-group-unh-q3-2023-earnings-call-trans/  works!
https://www.fool.com/earnings/call-transcripts/2023/10/13/jpmorgan-chase-jpm-q3-2023-earnings-call-transcrip/  works!
https://www.fool.com/earnings/call-transcripts/2023/10/12/commercial-metals-cmc-q4-2023-earnings-call-transc/  works!
https://www.fool.com/earnings/call-transcripts/2023/10/12/delta-air-lines-dal-q3-2023-earnings-call-transcri/  works!
https://www.fool.com/earnings/call-transcripts/2023/10/12/walgreens-

In [9]:
## Testing the dataframe
df

,Link,Ticker,RawContent
0,/earnings/call-transcripts/2023/10/13/oil-dri-...,ODC,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o..."
1,/earnings/call-transcripts/2023/10/13/citigrou...,C,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o..."
2,/earnings/call-transcripts/2023/10/13/wells-fa...,WFC,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o..."
3,/earnings/call-transcripts/2023/10/13/blackroc...,BLK,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o..."
4,/earnings/call-transcripts/2023/10/13/unitedhe...,UNH,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o..."
...,...,...,...
195,/earnings/call-transcripts/2023/08/07/palantir...,PLTR,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o..."
196,/earnings/call-transcripts/2023/08/07/skyworks...,SWKS,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o..."
197,/earnings/call-transcripts/2023/08/07/beyond-m...,BYND,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o..."
198,/earnings/call-transcripts/2023/08/07/paymentu...,PAY,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o..."


In [10]:
x= df['RawContent'].max() 
print(len(x))

180150


In [11]:
df.to_csv('transcripts_output.csv')

In [13]:
## Copying the dataframe to a new dataframe, so i don't change the content of the original. (I don't want to get the content again because the time of execution is 7 minutes)
dfwordcalc = pd.read_csv('transcripts_output.csv')

In [14]:
x = dfwordcalc['RawContent'].max() 
print(len(x))

180150


In [15]:
### Getting the content of the prepared remarks and putting it in a new column

## I found this sintax to apply a function with a dataframe column and variable, is interesting to see how it works
dfwordcalc['Raw_Prepared_Remarks'] = dfwordcalc['RawContent'].apply(lambda x: extract_text_between_tags(str(x), "<h2>Prepared Remarks:</h2>", "<h2>Questions &amp; Answers:</h2>"))
dfwordcalc['Content_Prepared_Remarks'] = dfwordcalc['Raw_Prepared_Remarks'].apply(lambda x: BeautifulSoup(str(x), 'html.parser').getText(separator=" "))

dfwordcalc

,Unnamed: 0,Link,Ticker,RawContent,Raw_Prepared_Remarks,Content_Prepared_Remarks
0,0,/earnings/call-transcripts/2023/10/12/commerci...,CMC,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Hel...,"[' Operator Hello, and welcome everyone to ..."
1,1,/earnings/call-transcripts/2023/10/12/delta-ai...,DAL,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,"[' Operator Good morning, everyone, and wel..."
2,2,/earnings/call-transcripts/2023/10/12/walgreen...,WBA,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,[' Operator Good morning. My name is Krista...
3,3,/earnings/call-transcripts/2023/10/11/voxx-int...,VOXX,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,"[' Operator Good day, and thank you for sta..."
4,4,/earnings/call-transcripts/2023/10/05/constell...,STZ,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Gre...,[' Operator Greetings. Welcome to the Const...
...,...,...,...,...,...,...
195,195,/earnings/call-transcripts/2023/09/01/mongodb-...,MDB,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,[' Operator Good day. Thank you for standin...
196,196,/earnings/call-transcripts/2023/09/01/broadcom...,AVGO,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Wel...,[' Operator Welcome to the Broadcom Inc.\'s...
197,197,/earnings/call-transcripts/2023/08/31/lululemo...,LULU,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Tha...,[' Operator Thank you for standing by. This...
198,198,/earnings/call-transcripts/2023/08/31/tillys-t...,TLYS,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Hel...,"[' Operator Hello, and welcome to Tilly\'s ..."


#### Calculate the same scores for each document that we've been calculating (see Helper file for more details)

In [16]:
### Importing the L_M positive and negative words list.

## Importing the list of words from excel file
## -> Insert the corect path where the file was saved
negWords = pd.read_excel("L_M_Dict\Lm_Negative.xlsx")
posWords = pd.read_excel("L_M_Dict\Lm_Positive.xlsx")

## Creating the list of positive and negative words 
posWordsList = [str(w).lower() for w in posWords['Word']]
negWordsList = [str(w).lower() for w in negWords['Word']]

## applying the stemm in the lists 
stemposWordsList = Helper.stemmed_base_list(posWordsList)
stemnegWordsList = Helper.stemmed_base_list(negWordsList)

In [17]:
### Applying the tone words calculations in the content on the dataframe

dfwordcalc['Content_Clear'] = dfwordcalc['Content_Prepared_Remarks'].apply(Helper.replace_punct)
dfwordcalc['Word_Count'] = dfwordcalc['Content_Clear'].apply(Helper.count_word)
dfwordcalc['Words_Dict'] = dfwordcalc['Content_Clear'].apply(Helper.dict_content)
dfwordcalc['Pos_Words'] = dfwordcalc.apply(lambda row: Helper.count_tone_words(row['Words_Dict'], posWordsList), axis=1)
dfwordcalc['Neg_Words'] = dfwordcalc.apply(lambda row: Helper.count_tone_words(row['Words_Dict'], negWordsList), axis=1)
dfwordcalc['%Pos_Words'] = dfwordcalc['Pos_Words']/dfwordcalc['Word_Count']
dfwordcalc['%Neg_Words'] = dfwordcalc['Neg_Words']/dfwordcalc['Word_Count']
dfwordcalc['Content_Stemmed'] = dfwordcalc['Content_Clear'].apply(Helper.stem_content)
dfwordcalc['Stemmed_Word_Dict'] = dfwordcalc['Words_Dict'] = dfwordcalc['Content_Clear'].apply(Helper.dict_content)
dfwordcalc['Stemm_Pos_Words'] = dfwordcalc.apply(lambda row: Helper.count_tone_words(row['Stemmed_Word_Dict'], stemposWordsList), axis=1)
dfwordcalc['Stemm_Neg_Wordst'] = dfwordcalc.apply(lambda row: Helper.count_tone_words(row['Stemmed_Word_Dict'], stemnegWordsList), axis=1)
dfwordcalc['%Pos_Words_Stemmed'] = dfwordcalc['Stemm_Pos_Words']/dfwordcalc['Word_Count']
dfwordcalc['%Neg_Words_Stemmed'] = dfwordcalc['Stemm_Neg_Wordst']/dfwordcalc['Word_Count']


dfwordcalc

,Unnamed: 0,Link,Ticker,RawContent,Raw_Prepared_Remarks,Content_Prepared_Remarks,Content_Clear,Word_Count,Words_Dict,Pos_Words,Neg_Words,%Pos_Words,%Neg_Words,Content_Stemmed,Stemmed_Word_Dict,Stemm_Pos_Words,Stemm_Neg_Wordst,%Pos_Words_Stemmed,%Neg_Words_Stemmed
0,0,/earnings/call-transcripts/2023/10/12/commerci...,CMC,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Hel...,"[' Operator Hello, and welcome everyone to ...",[ operator hello and welcome everyone to th...,4672,"{'[': 1, 'operator': 2, 'hello': 1, 'and': 155...",101,45,0.021618,0.009632,[ oper hello and welcom everyon to the fourth ...,"{'[': 1, 'operator': 2, 'hello': 1, 'and': 155...",50,46,0.010702,0.009846
1,1,/earnings/call-transcripts/2023/10/12/delta-ai...,DAL,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,"[' Operator Good morning, everyone, and wel...",[ operator good morning everyone and welcom...,2758,"{'[': 1, 'operator': 1, 'good': 5, 'morning': ...",62,21,0.022480,0.007614,[ oper good morn everyon and welcom to the del...,"{'[': 1, 'operator': 1, 'good': 5, 'morning': ...",39,24,0.014141,0.008702
2,2,/earnings/call-transcripts/2023/10/12/walgreen...,WBA,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,[' Operator Good morning. My name is Krista...,[ operator good morning my name is krista a...,5520,"{'[': 1, 'operator': 3, 'good': 6, 'morning': ...",133,48,0.024094,0.008696,[ oper good morn my name is krista and i will ...,"{'[': 1, 'operator': 3, 'good': 6, 'morning': ...",48,50,0.008696,0.009058
3,3,/earnings/call-transcripts/2023/10/11/voxx-int...,VOXX,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,"[' Operator Good day, and thank you for sta...",[ operator good day and thank you for stand...,3025,"{'[': 1, 'operator': 2, 'good': 5, 'day': 2, '...",47,51,0.015537,0.016860,[ oper good day and thank you for stand by wel...,"{'[': 1, 'operator': 2, 'good': 5, 'day': 2, '...",14,35,0.004628,0.011570
4,4,/earnings/call-transcripts/2023/10/05/constell...,STZ,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Gre...,[' Operator Greetings. Welcome to the Const...,[ operator greetings welcome to the constel...,4039,"{'[': 1, 'operator': 1, 'greetings': 1, 'welco...",80,38,0.019807,0.009408,[ oper greet welcom to the constel brand fisca...,"{'[': 1, 'operator': 1, 'greetings': 1, 'welco...",25,21,0.006190,0.005199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,/earnings/call-transcripts/2023/09/01/mongodb-...,MDB,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,[' Operator Good day. Thank you for standin...,[ operator good day thank you for standing ...,3990,"{'[': 1, 'operator': 2, 'good': 2, 'day': 1, '...",91,23,0.022807,0.005764,[ oper good day thank you for stand by welcom ...,"{'[': 1, 'operator': 2, 'good': 2, 'day': 1, '...",35,30,0.008772,0.007519
196,196,/earnings/call-transcripts/2023/09/01/broadcom...,AVGO,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Wel...,[' Operator Welcome to the Broadcom Inc.\'s...,[ operator welcome to the broadcom inc\s th...,2330,"{'[': 1, 'operator': 3, 'welcome': 1, 'to': 60...",22,14,0.009442,0.006009,[ oper welcom to the broadcom inc\ third quart...,"{'[': 1, 'operator': 3, 'welcome': 1, 'to': 60...",12,14,0.005150,0.006009
197,197,/earnings/call-transcripts/2023/08/31/lululemo...,LULU,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Tha...,[' Operator Thank you for standing by. This...,[ operator thank you for standing by this i...,4001,"{'[': 1, 'operator': 3, 'thank': 4, 'you': 6, ...",105,12,0.026243,0.002999,[ oper thank you for stand by thi is the confe

In [18]:
### Applying the redability and undersandability calculations in the content on the dataframe

dfwordcalc['n_Words_Tokenize'] = dfwordcalc['Content_Prepared_Remarks'].apply(Helper.count_word_with_tokenize)
dfwordcalc['Count_Sentences'] = dfwordcalc['Content_Prepared_Remarks'].apply(Helper.count_sentences)
dfwordcalc['Avg_Sentence_Length'] = dfwordcalc['n_Words_Tokenize']/dfwordcalc['Count_Sentences']
dfwordcalc['Complex_Words'] = dfwordcalc['Content_Clear'].apply(Helper.complex_words)
dfwordcalc['Rate_Complex_Words'] = dfwordcalc['Complex_Words']/dfwordcalc['n_Words_Tokenize']
dfwordcalc['FOG_Index'] = dfwordcalc.apply(lambda row: Helper.calc_fog_index(row['Avg_Sentence_Length'], row['Rate_Complex_Words']), axis=1)


In [19]:
## Checking the dataframe
dfwordcalc

,Unnamed: 0,Link,Ticker,RawContent,Raw_Prepared_Remarks,Content_Prepared_Remarks,Content_Clear,Word_Count,Words_Dict,Pos_Words,...,Stemm_Pos_Words,Stemm_Neg_Wordst,%Pos_Words_Stemmed,%Neg_Words_Stemmed,n_Words_Tokenize,Count_Sentences,Avg_Sentence_Length,Complex_Words,Rate_Complex_Words,FOG_Index
0,0,/earnings/call-transcripts/2023/10/12/commerci...,CMC,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Hel...,"[' Operator Hello, and welcome everyone to ...",[ operator hello and welcome everyone to th...,4672,"{'[': 1, 'operator': 2, 'hello': 1, 'and': 155...",101,...,50,46,0.010702,0.009846,4744,213,22.272300,1197,0.252319,9.009848
1,1,/earnings/call-transcripts/2023/10/12/delta-ai...,DAL,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,"[' Operator Good morning, everyone, and wel...",[ operator good morning everyone and welcom...,2758,"{'[': 1, 'operator': 1, 'good': 5, 'morning': ...",62,...,39,24,0.014141,0.008702,2793,156,17.903846,626,0.224132,7.251191
2,2,/earnings/call-transcripts/2023/10/12/walgreen...,WBA,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,[' Operator Good morning. My name is Krista...,[ operator good morning my name is krista a...,5520,"{'[': 1, 'operator': 3, 'good': 6, 'morning': ...",133,...,48,50,0.008696,0.009058,5581,288,19.378472,1283,0.229887,7.843344
3,3,/earnings/call-transcripts/2023/10/11/voxx-int...,VOXX,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,"[' Operator Good day, and thank you for sta...",[ operator good day and thank you for stand...,3025,"{'[': 1, 'operator': 2, 'good': 5, 'day': 2, '...",47,...,14,35,0.004628,0.011570,3064,143,21.426573,579,0.188969,8.646217
4,4,/earnings/call-transcripts/2023/10/05/constell...,STZ,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Gre...,[' Operator Greetings. Welcome to the Const...,[ operator greetings welcome to the constel...,4039,"{'[': 1, 'operator': 1, 'greetings': 1, 'welco...",80,...,25,21,0.006190,0.005199,4060,152,26.710526,943,0.232266,10.777117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,/earnings/call-transcripts/2023/09/01/mongodb-...,MDB,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,[' Operator Good day. Thank you for standin...,[ operator good day thank you for standing ...,3990,"{'[': 1, 'operator': 2, 'good': 2, 'day': 1, '...",91,...,35,30,0.008772,0.007519,4029,178,22.634831,977,0.242492,9.150929
196,196,/earnings/call-transcripts/2023/09/01/broadcom...,AVGO,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Wel...,[' Operator Welcome to the Broadcom Inc.\'s...,[ operator welcome to the broadcom inc\s th...,2330,"{'[': 1, 'operator': 3, 'welcome': 1, 'to': 60...",22,...,12,14,0.005150,0.006009,2348,124,18.935484,539,0.229557,7.666016
197,197,/earnings/call-transcripts/2023/08/31/lululemo...,LULU,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Tha...,[' Operator Thank you for standing by. This...,[ operator thank you for standing by this i...,4001,"{'[': 1, 'operator': 3, 'thank': 4, 'you': 6, ...",105,...,46,27,0.011497,0.006748,4059,206,19.703883,808,0.199064,7.961179
198,198,/earnings/call-transcripts/2023/08/31/tillys-t...,TLYS,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Hel...,"[' Operator Hello, and welcome to Tilly\'s ...",[ operator hello and welcome to tilly\s sec...,1680,"{'[': 1, 'operator': 2, 'hello': 1, 'and': 50,...",32,...,10,18,0.005952,0.010714,1704,69,24.695652,335,0.196596,9.956899


In [20]:
## Verifying the dataframe columns to extract the right ones corectly to a readable dataframe 
dfwordcalc.columns

Index(['Unnamed: 0', 'Link', 'Ticker', 'RawContent', 'Raw_Prepared_Remarks',
       'Content_Prepared_Remarks', 'Content_Clear', 'Word_Count', 'Words_Dict',
       'Pos_Words', 'Neg_Words', '%Pos_Words', '%Neg_Words', 'Content_Stemmed',
       'Stemmed_Word_Dict', 'Stemm_Pos_Words', 'Stemm_Neg_Wordst',
       '%Pos_Words_Stemmed', '%Neg_Words_Stemmed', 'n_Words_Tokenize',
       'Count_Sentences', 'Avg_Sentence_Length', 'Complex_Words',
       'Rate_Complex_Words', 'FOG_Index'],
      dtype='object')

In [21]:
## Creating a clear dataframe only with the columns I need
dfwordcalcclear = dfwordcalc[["Ticker","Word_Count", "Pos_Words","Neg_Words","%Pos_Words","%Neg_Words", "Stemm_Pos_Words", "Stemm_Neg_Wordst","%Pos_Words_Stemmed",
                   "%Neg_Words_Stemmed", "n_Words_Tokenize", "Complex_Words", "FOG_Index",]]

dfwordcalcclear

,Ticker,Word_Count,Pos_Words,Neg_Words,%Pos_Words,%Neg_Words,Stemm_Pos_Words,Stemm_Neg_Wordst,%Pos_Words_Stemmed,%Neg_Words_Stemmed,n_Words_Tokenize,Complex_Words,FOG_Index
0,CMC,4672,101,45,0.021618,0.009632,50,46,0.010702,0.009846,4744,1197,9.009848
1,DAL,2758,62,21,0.022480,0.007614,39,24,0.014141,0.008702,2793,626,7.251191
2,WBA,5520,133,48,0.024094,0.008696,48,50,0.008696,0.009058,5581,1283,7.843344
3,VOXX,3025,47,51,0.015537,0.016860,14,35,0.004628,0.011570,3064,579,8.646217
4,STZ,4039,80,38,0.019807,0.009408,25,21,0.006190,0.005199,4060,943,10.777117
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,MDB,3990,91,23,0.022807,0.005764,35,30,0.008772,0.007519,4029,977,9.150929
196,AVGO,2330,22,14,0.009442,0.006009,12,14,0.005150,0.006009,2348,539,7.666016
197,LULU,4001,105,12,0.026243,0.002999,46,27,0.011497,0.006748,4059,808,7.961179
198,TLYS,1680,32,22,0.019048,0.013095,10,18,0.005952,0.010714,1704,335,9.956899


#### Bônus: Separating the CEO and CFO speeches to calculate the previous things

### 4a. Testing the extration of CEO and CFO Speeches

In [23]:
## Extracting the prepared remarks from one row of the dataframe, is a very useful way to fix the bugs and find the right pattern
raw_content = str(dfwordcalc['RawContent'].loc[1])

print(raw_content)



<!DOCTYPE html>
<html lang="en" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">
<head>


<script type="text/javascript" src="https://cdn.cookielaw.org/consent/02abb198-81a8-49e5-a9b1-f69a5dd9c039/OtAutoBlock.js"></script>
<script src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" type="text/javascript" charset="UTF-8" data-domain-script="02abb198-81a8-49e5-a9b1-f69a5dd9c039"></script>

<script>
function OptanonWrapper(){var t;window.infotrackInitialized||void 0===window.Infotrack?window.Infotrack.setSegmentIntegrations():(window.infotrackInitialized=!0,Infotrack.load("https://g.foolcdn.com/mms/resources/js/infotrack-ot_min.js"),t=t||0,setTimeout(function(){window.Infotrack.initialize("usmf")},t))}window.top!=window&&(window.analytics=window.top.analytics,window.inIframe=!0);var segmentKey="CL257pIDks5ukrbYAs7Ez3fkLz1bWVnJ",segmentSnippetVersion="4.15.2",getSegmentUrl=function(t){return t=t||window.segmentKey,("https:"===document.l

In [25]:
### Getting the prepared remarks again

## As verifyed, the section always starts with "<h2>Prepared Remarks:</h2>" and ends with "<h2>Questions &amp; Answers:</h2>"
start_tag = "<h2>Prepared Remarks:</h2>"
end_tag = "Answers:</h2>"

## applying the function extract_text_between_tags in test page.
raw_prepared_remarks = extract_text_between_tags(raw_content, start_tag, end_tag)

## Join the content to be sure that is just one string
content_raw_prepared_remarks = ' '.join(raw_prepared_remarks)

## Printing the content to see the if is working
print(content_raw_prepared_remarks)

 <br/> <p><strong>Operator</strong></p><p>Good morning, everyone, and welcome to the Delta Air Lines September quarter 2023 financial results conference call. My name is Matthew, and I will be your coordinator. [Operator instructions] As a reminder, today's call is being recorded. [Operator instructions] I would now like to turn the conference over to Julie Stewart, vice president of investor relations.</p> <p>Please go ahead.</p><p><strong>Julie Stewart</strong> -- <em>Vice President, Investor Relations</em></p><div class="interad"></div> <p>Thank you, Matthew. Good morning, everyone, and thanks for joining us. Today, in Atlanta, we are joined by CEO, Ed Bastian; president, Glen Hauenstein; and our CFO, Dan Janki. Ed will open the call with an overview of Delta's performance and strategy.</p> <p>Glen will provide an update on the revenue environment, and Dan will discuss costs in our balance sheet. After the prepared remarks, we'll take analyst questions and then we'll move to our med

In [26]:
## As verifyed, the speeches always starts with "<em>Chief Executive Officer</em>" and ends with "</strong> --(somthing)"


## Creating the functions extract_speechs to get the content of the chiefs.
def get_cfo_speech (text) :
    cfo_pattern = "Chief Financial Officer\<\/em\>(.*?)\<\/strong\> \-\-|Chief Financial Officer\<\/em>(.*?)\<h2\>Questions|Chief Financial Officere\<\/em>(.*?)\<h2\>Questions"
    regEx = re.compile(cfo_pattern, flags=re.DOTALL | re.IGNORECASE)
    cfo_speech = regEx.findall(str(text))
    return cfo_speech
def get_ceo_speech (text) :
    ceo_pattern = "Chief Executive Officer\<\/em\>(.*?)\<\/strong\> \-\-|Chief Executive Officer\<\/em>(.*?)\<h2\>Questions|Chief Executive Officere\<\/em>(.*?)\<h2\>Questions"
    regEx = re.compile(ceo_pattern, flags=re.DOTALL | re.IGNORECASE)
    ceo_speech = regEx.findall(str(text))
    return ceo_speech

## Applying the functions in raw prepared remarks
cfo_speech = get_cfo_speech(content_raw_prepared_remarks)
ceo_speech = get_ceo_speech(content_raw_prepared_remarks)

## See if the functions are working
print(ceo_speech)
print(cfo_speech)



[('</p><div class="interad"></div> <p>Well, thank you, Julie, and good morning, everyone. We appreciate you joining us. Before we start, I want to acknowledge the unfolding war in Israel and the tragic loss of life that has ensued. Delta is donating $1 million to the American Red Cross for the International Committee of the Red Cross to help fund humanitarian efforts in the conflict.</p> <p>This includes emergency assistance such as health services, emergency care, ambulance services, and other critical needs. Our inbound and outbound flights to Tel Aviv have been suspended through October 31 to ensure the safety and security of our customers and employees. We\'re also offering a customer waiver for travel to Tel Aviv for those who need to change their travel plans. Our hearts are with everyone impacted by these tragic and horrific events.</p> <p>Turning to our news for the day. This morning, Delta reported September quarter results, posting earnings of $2.03 per share, a 35% increase 

In [27]:
### Aplying the speeches to the collunm in dataframe
dfSpeechers = pd.read_csv('transcripts_output.csv')
dfSpeechers['Raw_Prepared_Remarks'] = dfSpeechers['RawContent'].apply(lambda x: extract_text_between_tags(str(x), "<h2>Prepared Remarks:</h2>","Answers:</h2"))


In [28]:
dfSpeechers

,Unnamed: 0,Link,Ticker,RawContent,Raw_Prepared_Remarks
0,0,/earnings/call-transcripts/2023/10/12/commerci...,CMC,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Hel...
1,1,/earnings/call-transcripts/2023/10/12/delta-ai...,DAL,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...
2,2,/earnings/call-transcripts/2023/10/12/walgreen...,WBA,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...
3,3,/earnings/call-transcripts/2023/10/11/voxx-int...,VOXX,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...
4,4,/earnings/call-transcripts/2023/10/05/constell...,STZ,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Gre...
...,...,...,...,...,...
195,195,/earnings/call-transcripts/2023/09/01/mongodb-...,MDB,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...
196,196,/earnings/call-transcripts/2023/09/01/broadcom...,AVGO,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Wel...
197,197,/earnings/call-transcripts/2023/08/31/lululemo...,LULU,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Tha...
198,198,/earnings/call-transcripts/2023/08/31/tillys-t...,TLYS,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Hel...


In [29]:
### Applying the functions to get the speeches of each chief
dfSpeechers['CEO_Speech'] = dfSpeechers['Raw_Prepared_Remarks'].apply(get_ceo_speech)
dfSpeechers['CFO_Speech'] = dfSpeechers['Raw_Prepared_Remarks'].apply(get_cfo_speech)



In [30]:
## Print to see if is working
dfSpeechers

,Unnamed: 0,Link,Ticker,RawContent,Raw_Prepared_Remarks,CEO_Speech,CFO_Speech
0,0,/earnings/call-transcripts/2023/10/12/commerci...,CMC,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Hel...,"[(</p> <p>Thank you, Barbara. It\'s an honor t...","[(</p> <p>Thank you, Peter, and good morning t..."
1,1,/earnings/call-transcripts/2023/10/12/delta-ai...,DAL,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,"[(</p><div class=""interad""></div> <p>Well, tha...","[(</p> <p>Thank you, Glen, and good morning to..."
2,2,/earnings/call-transcripts/2023/10/12/walgreen...,WBA,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,"[(</p> <p>Thanks, Tiffany, and good morning, e...","[(</p> <p>Thank you, Ginger, and good morning...."
3,3,/earnings/call-transcripts/2023/10/11/voxx-int...,VOXX,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,"[(</p><div class=""interad""></div> <p>Thanks, G...","[(</p> <p>Thanks, Pat, and good morning, every..."
4,4,/earnings/call-transcripts/2023/10/05/constell...,STZ,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Gre...,"[(</p> <p>Thank you, Joe, and good morning, ev...","[(, </p> <p>Thank you, Bill, and good morning,..."
...,...,...,...,...,...,...,...
195,195,/earnings/call-transcripts/2023/09/01/mongodb-...,MDB,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,"[(</p> <p>Thank you, Brian, and thank you to e...",[]
196,196,/earnings/call-transcripts/2023/09/01/broadcom...,AVGO,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Wel...,"[(</p><div class=""interad""></div> <p>Thank you...","[(, </p> <p>Thank you, Hock. Let me now provid..."
197,197,/earnings/call-transcripts/2023/08/31/lululemo...,LULU,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Tha...,"[(</p> <p>Thank you, Howard. I\'m pleased to b...","[(</p> <p>Thanks, Calvin. Our momentum remaine..."
198,198,/earnings/call-transcripts/2023/08/31/tillys-t...,TLYS,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Hel...,"[(</p><div class=""interad""></div> <p>Thanks, G...",[]


In [31]:
### Now i deleted the empty rows because i don't found the pattern, but with more time is possible to find the pattern to the empty rows 

dfSpeechers = dfSpeechers[(dfSpeechers['CEO_Speech'].str.len() != 0) & (dfSpeechers['CFO_Speech'].str.len() != 0)]

In [32]:
### Checking the dataframe to see how much was extracted

dfSpeechers

,Unnamed: 0,Link,Ticker,RawContent,Raw_Prepared_Remarks,CEO_Speech,CFO_Speech
0,0,/earnings/call-transcripts/2023/10/12/commerci...,CMC,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Hel...,"[(</p> <p>Thank you, Barbara. It\'s an honor t...","[(</p> <p>Thank you, Peter, and good morning t..."
1,1,/earnings/call-transcripts/2023/10/12/delta-ai...,DAL,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,"[(</p><div class=""interad""></div> <p>Well, tha...","[(</p> <p>Thank you, Glen, and good morning to..."
2,2,/earnings/call-transcripts/2023/10/12/walgreen...,WBA,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,"[(</p> <p>Thanks, Tiffany, and good morning, e...","[(</p> <p>Thank you, Ginger, and good morning...."
3,3,/earnings/call-transcripts/2023/10/11/voxx-int...,VOXX,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,"[(</p><div class=""interad""></div> <p>Thanks, G...","[(</p> <p>Thanks, Pat, and good morning, every..."
4,4,/earnings/call-transcripts/2023/10/05/constell...,STZ,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Gre...,"[(</p> <p>Thank you, Joe, and good morning, ev...","[(, </p> <p>Thank you, Bill, and good morning,..."
...,...,...,...,...,...,...,...
193,193,/earnings/call-transcripts/2023/09/01/sentinel...,S,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,"[(</p> <p>Good afternoon, everyone, and thank ...","[(, </p> <p>Thank you, Tomer. Today, I\'ll dis..."
194,194,/earnings/call-transcripts/2023/09/01/elastic-...,ESTC,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Goo...,"[(</p> <p>Thank you, Janice, and thank you all...","[(, </p> <p>Thanks, Ash. We are very pleased w..."
196,196,/earnings/call-transcripts/2023/09/01/broadcom...,AVGO,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Wel...,"[(</p><div class=""interad""></div> <p>Thank you...","[(, </p> <p>Thank you, Hock. Let me now provid..."
197,197,/earnings/call-transcripts/2023/08/31/lululemo...,LULU,"\n\n<!DOCTYPE html>\n<html lang=""en"" prefix=""o...",[ <br/> <p><strong>Operator</strong></p><p>Tha...,"[(</p> <p>Thank you, Howard. I\'m pleased to b...","[(</p> <p>Thanks, Calvin. Our momentum remaine..."


In [33]:
### Creating to separete dataframes to analyze the speeches of each chief separately

ceo_analisys_df = dfSpeechers[['Link', 'Ticker','CEO_Speech']]
cfo_analisys_df = dfSpeechers[['Link', 'Ticker','CFO_Speech']]

In [34]:
### Applying the tone words calculations in the content on the dataframe


ceo_analisys_df['Content'] = ceo_analisys_df['CEO_Speech'].apply(lambda x: BeautifulSoup(str(x), 'html.parser').getText(separator=" "))
ceo_analisys_df['Content_Clear'] = ceo_analisys_df['Content'].apply(Helper.replace_punct)
ceo_analisys_df['Word_Count'] = ceo_analisys_df['Content_Clear'].apply(Helper.count_word)
ceo_analisys_df['Words_Dict'] = ceo_analisys_df['Content_Clear'].apply(Helper.dict_content)
ceo_analisys_df['Pos_Words'] = ceo_analisys_df.apply(lambda row: Helper.count_tone_words(row['Words_Dict'], posWordsList), axis=1)
ceo_analisys_df['Neg_Words'] = ceo_analisys_df.apply(lambda row: Helper.count_tone_words(row['Words_Dict'], negWordsList), axis=1)
ceo_analisys_df['%Pos_Words'] = ceo_analisys_df['Pos_Words']/ceo_analisys_df['Word_Count']
ceo_analisys_df['%Neg_Words'] = ceo_analisys_df['Neg_Words']/ceo_analisys_df['Word_Count']
ceo_analisys_df['Content_Stemmed'] = ceo_analisys_df['Content_Clear'].apply(Helper.stem_content)
ceo_analisys_df['Stemmed_Word_Dict'] = ceo_analisys_df['Words_Dict'] = ceo_analisys_df['Content_Clear'].apply(Helper.dict_content)
ceo_analisys_df['Stemm_Pos_Words'] = ceo_analisys_df.apply(lambda row: Helper.count_tone_words(row['Stemmed_Word_Dict'], stemposWordsList), axis=1)
ceo_analisys_df['Stemm_Neg_Wordst'] = ceo_analisys_df.apply(lambda row: Helper.count_tone_words(row['Stemmed_Word_Dict'], stemnegWordsList), axis=1)
ceo_analisys_df['%Pos_Words_Stemmed'] = ceo_analisys_df['Stemm_Pos_Words']/ceo_analisys_df['Word_Count']
ceo_analisys_df['%Neg_Words_Stemmed'] = ceo_analisys_df['Stemm_Neg_Wordst']/ceo_analisys_df['Word_Count']


ceo_analisys_df

C:\Users\rafae\AppData\Local\Temp\ipykernel_1480\575491863.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ceo_analisys_df['Content'] = ceo_analisys_df['CEO_Speech'].apply(lambda x: BeautifulSoup(str(x), 'html.parser').getText(separator=" "))
C:\Users\rafae\AppData\Local\Temp\ipykernel_1480\575491863.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ceo_analisys_df['Content_Clear'] = ceo_analisys_df['Content'].apply(Helper.replace_punct)
C:\Users\rafae\AppData\Local\Temp\ipykernel_1480\575491863.py:6:

,Link,Ticker,CEO_Speech,Content,Content_Clear,Word_Count,Words_Dict,Pos_Words,Neg_Words,%Pos_Words,%Neg_Words,Content_Stemmed,Stemmed_Word_Dict,Stemm_Pos_Words,Stemm_Neg_Wordst,%Pos_Words_Stemmed,%Neg_Words_Stemmed
0,/earnings/call-transcripts/2023/10/12/commerci...,CMC,"[(</p> <p>Thank you, Barbara. It\'s an honor t...","[("" Thank you, Barbara. It\\'s an honor to t...","["" thank you barbara it\\s an honor to take ...",2276,"{'[""': 1, 'thank': 4, 'you': 3, 'barbara': 1, ...",44,23,0.019332,0.010105,"["" thank you barbara it\\ an honor to take the...","{'[""': 1, 'thank': 4, 'you': 3, 'barbara': 1, ...",24,19,0.010545,0.008348
1,/earnings/call-transcripts/2023/10/12/delta-ai...,DAL,"[(</p><div class=""interad""></div> <p>Well, tha...","[(' Well, thank you, Julie, and good morning...",[ well thank you julie and good morning ever...,965,"{'[': 1, 'well': 2, 'thank': 2, 'you': 3, 'jul...",21,9,0.021762,0.009326,[ well thank you juli and good morn everyon we...,"{'[': 1, 'well': 2, 'thank': 2, 'you': 3, 'jul...",15,10,0.015544,0.010363
2,/earnings/call-transcripts/2023/10/12/walgreen...,WBA,"[(</p> <p>Thanks, Tiffany, and good morning, e...","[("" Thanks, Tiffany, and good morning, every...","["" thanks tiffany and good morning everyone ...",1658,"{'[""': 1, 'thanks': 3, 'tiffany': 1, 'and': 80...",47,14,0.028347,0.008444,"["" thank tiffani and good morn everyon the goo...","{'[""': 1, 'thanks': 3, 'tiffany': 1, 'and': 80...",12,8,0.007238,0.004825
3,/earnings/call-transcripts/2023/10/11/voxx-int...,VOXX,"[(</p><div class=""interad""></div> <p>Thanks, G...","[(' Thanks, Glenn, and good morning, everyon...",[ thanks glenn and good morning everyone not...,1876,"{'[': 1, 'thanks': 1, 'glenn': 1, 'and': 81, '...",30,20,0.015991,0.010661,[ thank glenn and good morn everyon not much h...,"{'[': 1, 'thanks': 1, 'glenn': 1, 'and': 81, '...",8,19,0.004264,0.010128
4,/earnings/call-transcripts/2023/10/05/constell...,STZ,"[(</p> <p>Thank you, Joe, and good morning, ev...","[(' Thank you, Joe, and good morning, everyo...",[ thank you joe and good morning everyone we...,1618,"{'[': 1, 'thank': 1, 'you': 4, 'joe': 1, 'and'...",42,16,0.025958,0.009889,[ thank you joe and good morn everyon welcom t...,"{'[': 1, 'thank': 1, 'you': 4, 'joe': 1, 'and'...",10,6,0.006180,0.003708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,/earnings/call-transcripts/2023/09/01/sentinel...,S,"[(</p> <p>Good afternoon, everyone, and thank ...","[(' Good afternoon, everyone, and thank you ...",[ good afternoon everyone and thank you for ...,2215,"{'[': 1, 'good': 1, 'afternoon': 1, 'everyone'...",77,40,0.034763,0.018059,[ good afternoon everyon and thank you for joi...,"{'[': 1, 'good': 1, 'afternoon': 1, 'everyone'...",31,11,0.013995,0.004966
194,/earnings/call-transcripts/2023/09/01/elastic-...,ESTC,"[(</p> <p>Thank you, Janice, and thank you all...","[(' Thank you, Janice, and thank you all for...",[ thank you janice and thank you all for joi...,1865,"{'[': 1, 'thank': 4, 'you': 2, 'janice': 1, 'a...",55,10,0.029491,0.005362,[ thank you janic and thank you all for join u...,"{'[': 1, 'thank': 4, 'you': 2, 'janice': 1, 'a...",18,6,0.009651,0.003217
196,/earnings/call-transcripts/2023/09/01/broadcom...,AVGO,"[(</p><div class=""interad""></div> <p>Thank you...","[(' Thank you, Ji, and thank you, everyone, ...",[ thank you ji and thank you everyone for jo...,1296,"{'[': 1, 'thank': 2, 'you': 6, 'ji': 1, 'and':...",19,8,0.014660,0.006173,[ thank you ji and thank you everyon for join ...,"{'[': 1, 'thank': 2, 'you': 6, 'ji': 1, 'and':...",8,8,0.006173,0.006173
197,/earnings/call-transcripts/2023/08/31/lululemo...,LULU,"[(</p> <p>Thank you, Howard. I\'m pleased to b...","[(' Thank you, Howard. I\\\'m pleased to be ...",[ thank you howard i\\\m pleased to be here ...,2064,"{'[': 1, 'thank': 2, 'you': 3, 'howard': 1, 'i...",75,3,0.036337,0.001453,[ thank you howard i\\\m pleas to be here with...,"{'[': 1, 'thank': 2, 'you': 3, 'howard': 1, 'i.

In [35]:
### Applying the redability and undersandability calculations in the content on the dataframe

ceo_analisys_df['n_Words_Tokenize'] = ceo_analisys_df['Content'].apply(Helper.count_word_with_tokenize)
ceo_analisys_df['Count_Sentences'] = ceo_analisys_df['Content'].apply(Helper.count_sentences)
ceo_analisys_df['Avg_Sentence_Length'] = ceo_analisys_df['n_Words_Tokenize']/ceo_analisys_df['Count_Sentences']
ceo_analisys_df['Complex_Words'] = ceo_analisys_df['Content_Clear'].apply(Helper.complex_words)
ceo_analisys_df['Rate_Complex_Words'] = ceo_analisys_df['Complex_Words']/ceo_analisys_df['n_Words_Tokenize']
ceo_analisys_df['FOG_Index'] = ceo_analisys_df.apply(lambda row: Helper.calc_fog_index(row['Avg_Sentence_Length'], row['Rate_Complex_Words']), axis=1)


In [36]:
### Applying the tone words calculations in the content on the dataframe


cfo_analisys_df['Content'] = cfo_analisys_df['CFO_Speech'].apply(lambda x: BeautifulSoup(str(x), 'html.parser').getText(separator=" "))
cfo_analisys_df['Content_Clear'] = cfo_analisys_df['Content'].apply(Helper.replace_punct)
cfo_analisys_df['Word_Count'] = cfo_analisys_df['Content_Clear'].apply(Helper.count_word)
cfo_analisys_df['Words_Dict'] = cfo_analisys_df['Content_Clear'].apply(Helper.dict_content)
cfo_analisys_df['Pos_Words'] = cfo_analisys_df.apply(lambda row: Helper.count_tone_words(row['Words_Dict'], posWordsList), axis=1)
cfo_analisys_df['Neg_Words'] = cfo_analisys_df.apply(lambda row: Helper.count_tone_words(row['Words_Dict'], negWordsList), axis=1)
cfo_analisys_df['%Pos_Words'] = cfo_analisys_df['Pos_Words']/cfo_analisys_df['Word_Count']
cfo_analisys_df['%Neg_Words'] = cfo_analisys_df['Neg_Words']/cfo_analisys_df['Word_Count']
cfo_analisys_df['Content_Stemmed'] = cfo_analisys_df['Content_Clear'].apply(Helper.stem_content)
cfo_analisys_df['Stemmed_Word_Dict'] = cfo_analisys_df['Words_Dict'] = cfo_analisys_df['Content_Clear'].apply(Helper.dict_content)
cfo_analisys_df['Stemm_Pos_Words'] = cfo_analisys_df.apply(lambda row: Helper.count_tone_words(row['Stemmed_Word_Dict'], stemposWordsList), axis=1)
cfo_analisys_df['Stemm_Neg_Wordst'] = cfo_analisys_df.apply(lambda row: Helper.count_tone_words(row['Stemmed_Word_Dict'], stemnegWordsList), axis=1)
cfo_analisys_df['%Pos_Words_Stemmed'] = cfo_analisys_df['Stemm_Pos_Words']/cfo_analisys_df['Word_Count']
cfo_analisys_df['%Neg_Words_Stemmed'] = cfo_analisys_df['Stemm_Neg_Wordst']/cfo_analisys_df['Word_Count']


cfo_analisys_df

C:\Users\rafae\AppData\Local\Temp\ipykernel_1480\1568759122.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cfo_analisys_df['Content'] = cfo_analisys_df['CFO_Speech'].apply(lambda x: BeautifulSoup(str(x), 'html.parser').getText(separator=" "))
C:\Users\rafae\AppData\Local\Temp\ipykernel_1480\1568759122.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cfo_analisys_df['Content_Clear'] = cfo_analisys_df['Content'].apply(Helper.replace_punct)
C:\Users\rafae\AppData\Local\Temp\ipykernel_1480\1568759122.py

,Link,Ticker,CFO_Speech,Content,Content_Clear,Word_Count,Words_Dict,Pos_Words,Neg_Words,%Pos_Words,%Neg_Words,Content_Stemmed,Stemmed_Word_Dict,Stemm_Pos_Words,Stemm_Neg_Wordst,%Pos_Words_Stemmed,%Neg_Words_Stemmed
0,/earnings/call-transcripts/2023/10/12/commerci...,CMC,"[(</p> <p>Thank you, Peter, and good morning t...","[("" Thank you, Peter, and good morning to ev...","["" thank you peter and good morning to every...",1156,"{'[""': 1, 'thank': 1, 'you': 1, 'peter': 3, 'a...",26,16,0.022491,0.013841,"["" thank you peter and good morn to everyon on...","{'[""': 1, 'thank': 1, 'you': 1, 'peter': 3, 'a...",11,7,0.009516,0.006055
1,/earnings/call-transcripts/2023/10/12/delta-ai...,DAL,"[(</p> <p>Thank you, Glen, and good morning to...","[("" Thank you, Glen, and good morning to eve...","["" thank you glen and good morning to everyo...",709,"{'[""': 1, 'thank': 2, 'you': 1, 'glen': 1, 'an...",17,8,0.023977,0.011283,"["" thank you glen and good morn to everyon for...","{'[""': 1, 'thank': 2, 'you': 1, 'glen': 1, 'an...",8,4,0.011283,0.005642
2,/earnings/call-transcripts/2023/10/12/walgreen...,WBA,"[(</p> <p>Thank you, Ginger, and good morning....","[("" Thank you, Ginger, and good morning. Fou...","["" thank you ginger and good morning fourth-...",2236,"{'[""': 1, 'thank': 1, 'you': 4, 'ginger': 3, '...",38,28,0.016995,0.012522,"["" thank you ginger and good morn fourth-quart...","{'[""': 1, 'thank': 1, 'you': 4, 'ginger': 3, '...",14,24,0.006261,0.010733
3,/earnings/call-transcripts/2023/10/11/voxx-int...,VOXX,"[(</p> <p>Thanks, Pat, and good morning, every...","[("" Thanks, Pat, and good morning, everyone....","["" thanks pat and good morning everyone we r...",772,"{'[""': 1, 'thanks': 1, 'pat': 7, 'and': 28, 'g...",12,30,0.015544,0.038860,"["" thank pat and good morn everyon we report t...","{'[""': 1, 'thanks': 1, 'pat': 7, 'and': 28, 'g...",1,13,0.001295,0.016839
4,/earnings/call-transcripts/2023/10/05/constell...,STZ,"[(, </p> <p>Thank you, Bill, and good morning,...","[('', "" Thank you, Bill, and good morning, e...","[ "" thank you bill and good morning everyone...",2095,"{'[': 1, '""': 2, 'thank': 1, 'you': 3, 'bill':...",35,21,0.016706,0.010024,"[ "" thank you bill and good morn everyon our s...","{'[': 1, '""': 2, 'thank': 1, 'you': 3, 'bill':...",12,11,0.005728,0.005251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,/earnings/call-transcripts/2023/09/01/sentinel...,S,"[(, </p> <p>Thank you, Tomer. Today, I\'ll dis...","[('', "" Thank you, Tomer. Today, I\\'ll disc...","[ "" thank you tomer today i\\ll discuss our ...",1179,"{'[': 1, '""': 2, 'thank': 2, 'you': 2, 'tomer'...",40,9,0.033927,0.007634,"[ "" thank you tomer today i\\ll discuss our qu...","{'[': 1, '""': 2, 'thank': 2, 'you': 2, 'tomer'...",11,5,0.009330,0.004241
194,/earnings/call-transcripts/2023/09/01/elastic-...,ESTC,"[(, </p> <p>Thanks, Ash. We are very pleased w...","[('', "" Thanks, Ash. We are very pleased wit...","[ "" thanks ash we are very pleased with the ...",1470,"{'[': 1, '""': 2, 'thanks': 1, 'ash': 2, 'we': ...",32,6,0.021769,0.004082,"[ "" thank ash we are veri pleas with the stron...","{'[': 1, '""': 2, 'thanks': 1, 'ash': 2, 'we': ...",13,13,0.008844,0.008844
196,/earnings/call-transcripts/2023/09/01/broadcom...,AVGO,"[(, </p> <p>Thank you, Hock. Let me now provid...","[('', ' Thank you, Hock. Let me now provide ...",[ thank you hock let me now provide additio...,660,"{'[': 1, 'thank': 1, 'you': 1, 'hock': 1, 'let...",0,4,0.000000,0.006061,[ thank you hock let me now provid addit detai...,"{'[': 1, 'thank': 1, 'you': 1, 'hock': 1, 'let...",1,1,0.001515,0.001515
197,/earnings/call-transcripts/2023/08/31/lululemo...,LULU,"[(</p> <p>Thanks, Calvin. Our momentum remaine...","[("" Thanks, Calvin. Our momentum remained st...","["" thanks calvin our momentum remained stron...",1439,"{'[""': 1, 'thanks': 1, 'calvin': 3, 'our': 39,...",24,3,0.016678,0.002085,"["" thank calvin our momentum remain strong in ...","{'[""':

In [37]:
### Applying the redability and undersandability calculations in the content on the dataframe

cfo_analisys_df['n_Words_Tokenize'] = cfo_analisys_df['Content'].apply(Helper.count_word_with_tokenize)
cfo_analisys_df['Count_Sentences'] = cfo_analisys_df['Content'].apply(Helper.count_sentences)
cfo_analisys_df['Avg_Sentence_Length'] = cfo_analisys_df['n_Words_Tokenize']/cfo_analisys_df['Count_Sentences']
cfo_analisys_df['Complex_Words'] = cfo_analisys_df['Content_Clear'].apply(Helper.complex_words)
cfo_analisys_df['Rate_Complex_Words'] = cfo_analisys_df['Complex_Words']/cfo_analisys_df['n_Words_Tokenize']
cfo_analisys_df['FOG_Index'] = cfo_analisys_df.apply(lambda row: Helper.calc_fog_index(row['Avg_Sentence_Length'], row['Rate_Complex_Words']), axis=1)


In [41]:
ceo_analisys_df['Pos_Words'].describe()

count    168.000000
mean      54.583333
std       22.996073
min       19.000000
25%       41.750000
50%       53.000000
75%       61.000000
max      163.000000
Name: Pos_Words, dtype: float64

In [40]:
cfo_analisys_df['Pos_Words'].describe()

count    168.000000
mean      20.470238
std       12.958213
min        0.000000
25%       14.000000
50%       16.500000
75%       32.000000
max       88.000000
Name: Pos_Words, dtype: float64